In [29]:
import pandas as pd
import pandas_profiling as pprof
import matplotlib.pyplot as plt
import numpy as np

# Generate Pandas Profile

In [30]:
FEAT_LOC = '../3_generate_features/final_feature_array.csv'
LABEL_LOC = '../3_generate_features/final_label_array.csv'
feat_df = pd.read_csv(FEAT_LOC)
label_df = pd.read_csv(LABEL_LOC)
combined_df = pd.concat([feat_df, label_df],axis=1)

In [22]:
pprof.ProfileReport(combined_df, minimal=True).to_file('profile_minimal_combined.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Examine class frequencies

In [47]:
uniq, counts = np.unique(label_df.Prototype, return_counts=True)
freq_df = pd.DataFrame(np.transpose([uniq, counts]), columns=['Prototype','Count']).sort_values(by='Count', ascending=False)
freq_df['Count'] = freq_df['Count'].astype(int)

subdf = freq_df[freq_df.Count > 1]

fig, ax = plt.subplots()
x = subdf.Count.to_numpy()
y = subdf.Prototype.to_numpy()
plt.barh(subdf.Prototype, subdf.Count, color='teal')
plt.xlabel('Population')
# plt.ylabel('Prototype')
# plt.yticks(ticks=None)
ax.set_yticklabels([])
# plt.xscale('log')
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False)
# plt.show()
plt.savefig('../figures/proto_pop_bar.png', bbox_inches='tight', dpi=400)

In [46]:
fig, ax = plt.subplots()
plt.hist(freq_df.Count, bins=500)
# plt.yscale('log')
plt.ylabel('Number of Unique Prototypes')
plt.xlabel('Prototype Population')
plt.show()
plt.savefig('../figures/proto_hist.png', bbox_inches='tight', dpi=400)

/tmp/ipykernel_292/980337125.py:6: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [42]:
### Create reduced/simplified counts 

counts = freq_df.Count.to_numpy()
labels = freq_df.Prototype.to_numpy()
cutoff = 100
head_df = freq_df[freq_df.Count >= cutoff]
tail_df = freq_df[freq_df.Count < cutoff]
compress_df = pd.DataFrame({freq_df.columns[0]:[f'{len(tail_df)} prototypes ($N \leq {cutoff}$)'], 
                            freq_df.columns[1]:tail_df.Count.sum()})
reduced_df = pd.concat([head_df, compress_df])

fig, ax = plt.subplots(figsize=(8,7))
n_expl = 3
n_notexpl = (len(reduced_df)-n_expl)
plt.pie(reduced_df.Count,
        explode=[True]*n_expl + [False]*n_notexpl, 
        labels=reduced_df.Prototype)
plt.savefig('../figures/proto_pie.png', bbox_inches='tight', dpi=400)
# plt.show()

# Pairwise distributions